<a href="https://colab.research.google.com/github/jems2019/ReinforcementBasedTradingPlatform/blob/master/notebooks/Scraping_news_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install bs4

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sent_analysis = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [0]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint
import pdb

In [0]:
class SentimentAnalysis(object):

  def __init__(self, stock='apple'):
    self.url = 'https://www.businesstimes.com.sg/search/'+ str(stock) + '?page='
    self.sentiment_dates = {}

  def get_sentiment(self):
    """
    Function that scrapes the articles of businesstimes.com/<stock> and parses 
    through the articles and using a simple vader lexicon sentiment analysis
    library is able to come up with a sentiment score for the given article on 
    a given day. 
    If the sentimnet score is negative then the overall atricles for that day
    are negative.
    If the sentiment score is positive then the overall articles for that day
    are positive.  
    """
    for i in range(1,2):
        page = urlopen(self.url + str(i)).read()
        scrape = BeautifulSoup(page, features="html.parser")
        articles = scrape.findAll("div", {"class": "media-body"})
        print(len(articles))
        print (i)
        for post in articles:
            time.sleep(1)
            url = post.a['href']
            date = post.time.text
            print(date, url)
            try:
                page_link = urlopen(url).read()
            except:
                url = url[:-1]
                page_link = urlopen(url).read()
            soup = BeautifulSoup(page_link)
            info = soup.findAll("p")
            text_string = ""
            for sentence in info:
                text_string += sentence.text
            sentiment_score = sent_analysis.polarity_scores(text_string)['compound']
            self.sentiment_dates.setdefault(date, []).append(sentiment_score)

    self.date_of_sentiment = {}

    for a,b in self.sentiment_dates.items():
        self.date_of_sentiment[datetime.strptime(a, '%d %b %Y').date() 
        + timedelta(days=1)] = round(sum(b)/float(len(b)),2)

    pub_date = min(self.date_of_sentiment.keys())

    return self.date_of_sentiment

In [0]:
scores = SentimentAnalysis()

In [0]:
data = scores.get_sentiment()

10
1
07 Nov 2019 https://www.businesstimes.com.sg/technology/qualcomms-licensing-forecast-helped-by-apple-deal-drives-share-gains
05 Nov 2019 https://www.businesstimes.com.sg/real-estate/apple-pledges-us25b-for-affordable-california-housing
01 Nov 2019 https://www.businesstimes.com.sg/technology/ex-apple-executive-joins-startup-aimed-at-banishing-smartphone-cables
01 Nov 2019 https://www.businesstimes.com.sg/lifestyle/weekend-interview/tim-kobe-designer-of-apple-store-ceo-of-eight-inc
31 Oct 2019 https://www.businesstimes.com.sg/technology/apple-beats-profit-forecasts-amid-growth-in-services-wearables
31 Oct 2019 https://www.businesstimes.com.sg/technology/gender-neutral-emojis-hit-screens-in-new-apple-update-0
30 Oct 2019 https://www.businesstimes.com.sg/technology/gender-neutral-emojis-hit-screens-in-new-apple-update
29 Oct 2019 https://www.businesstimes.com.sg/real-estate/lego-builds-a-new-headquarters-to-compete-with-apple-and-nike
18 Oct 2019 https://www.businesstimes.com.sg/techn

In [0]:
data

{datetime.date(2019, 10, 18): 0.85,
 datetime.date(2019, 10, 19): 0.14,
 datetime.date(2019, 10, 30): 0.98,
 datetime.date(2019, 10, 31): 0.29,
 datetime.date(2019, 11, 1): 0.64,
 datetime.date(2019, 11, 2): 0.99,
 datetime.date(2019, 11, 6): 0.96,
 datetime.date(2019, 11, 8): 0.98}